In [ ]:
!pip install opacus

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.7/251.7 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 90.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import copy
import time
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from cryptography.fernet import Fernet  # Secure Aggregation
from torch.optim.lr_scheduler import ReduceLROnPlateau


In [ ]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.preprocessing import StandardScaler, LabelEncoder


device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
#   IoHT-Specific Clients & File Paths
file_paths = {
    "Wearable Medical Devices": "/content/Tuesday-WorkingHours.pcap_ISCX.csv",
    "Hospital Gateways": "/content/Wednesday-workingHours.pcap_ISCX.csv",
    "Industrial IoHT Systems": "/content/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv",
    "Smart Health Monitoring": "/content/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv",
    "Cloud-Based EHR Systems": "/content/Friday-WorkingHours-Morning.pcap_ISCX.csv",
}

client_data = {}

#   IoHT-Specific Attack Mapping
attack_mapping = {
    "BENIGN": "Normal",
    "FTP-Patator": "IoHT Credential Stuffing",
    "SSH-Patator": "IoHT Unauthorized Access",
    "DoS slowloris": "IoHT Service Degradation",
    "DoS Slowhttptest": "IoHT Service Degradation",
    "DoS Hulk": "IoHT DDoS Attack",
    "DoS GoldenEye": "IoHT DDoS Attack",
    "Heartbleed": "IoHT Data Leak",
    "DDoS": "IoHT Botnet Attack",
    "PortScan": "IoHT Reconnaissance",
    "Bot": "IoHT Malicious Bot Activity",
}

# Read & Process Each File
for client, file in file_paths.items():
    #print(f" Processing {client} dataset...")
    df = pd.read_csv(file)

    #  Preprocessing Steps
    df.columns = df.columns.str.strip()
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.dropna(inplace=True)

    #  Map Labels to IoHT-Specific Names
    df["Label"] = df["Label"].map(attack_mapping).fillna("Unknown Threat")

    client_data[client] = df

print(" IoHT-Specific Label Mapping Applied Successfully!")

#  Global Label Encoding
all_labels = np.concatenate([df["Label"].values for df in client_data.values()])
label_encoder = LabelEncoder()
label_encoder.fit(all_labels)

#  Convert Data to PyTorch Dataloaders
client_datasets = {}
feature_columns = None  # Store feature columns globally

for client, df in client_data.items():
    feature_columns = [col for col in df.columns if col != "Label"]
    X = df[feature_columns].values
    y = label_encoder.transform(df["Label"])

    #  Normalize Features
    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    #  Convert to Tensors
    X_tensor = torch.tensor(X, dtype=torch.float32)
    y_tensor = torch.tensor(y, dtype=torch.long)

    #  Create Train-Test Split
    dataset = TensorDataset(X_tensor, y_tensor)
    train_size = int(0.8 * len(dataset))
    test_size = len(dataset) - train_size
    train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

    client_datasets[client] = (train_dataset, test_dataset)

#  Create Dataloaders
client_train_loaders = {
    client: DataLoader(ds[0], batch_size=64, shuffle=True, num_workers=2, pin_memory=True)
    for client, ds in client_datasets.items()
}

client_test_loaders = {
    client: DataLoader(ds[1], batch_size=64, shuffle=False, num_workers=2, pin_memory=True)
    for client, ds in client_datasets.items()
}

print(" Data Preprocessing Completed Successfully!")


 IoHT-Specific Label Mapping Applied Successfully!
 Data Preprocessing Completed Successfully!


In [ ]:
#  Compute Class Weights

def compute_class_weights(labels, num_classes):
    class_counts = np.bincount(labels, minlength=num_classes)
    total_samples = sum(class_counts)

    class_weights = [
        (total_samples / (num_classes * count)) if count > 0 else (total_samples / (num_classes * 100))
        for count in class_counts
    ]

    class_weights = torch.tensor(class_weights, dtype=torch.float32, device=device)
    class_weights = torch.clamp(class_weights, min=0.1, max=3.0)  # Adjusted max weight

    return class_weights

In [ ]:
def initialize_weights(m):
    if isinstance(m, torch.nn.Linear) or isinstance(m, torch.nn.Conv1d):
        torch.nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            torch.nn.init.zeros_(m.bias)

In [ ]:
class CNN_LSTM(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(CNN_LSTM, self).__init__()

        self.conv1 = nn.Conv1d(in_channels=1, out_channels=64, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm1d(64)
        self.conv2 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm1d(128)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)  #  Reduced dropout

        self.lstm = nn.LSTM(input_size=128, hidden_size=256, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(256 * 2, num_classes)

    def forward(self, x):
        x = x.unsqueeze(1)
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.dropout(x)

        x = x.permute(0, 2, 1)
        x, _ = self.lstm(x)
        x = self.fc(x[:, -1, :])

        return x

In [ ]:
#  FedProx Loss Function
class FedProxLoss(nn.Module):
    def __init__(self, mu=0.05):
        super(FedProxLoss, self).__init__()
        self.mu = mu
        self.ce_loss = nn.CrossEntropyLoss()

    def forward(self, local_params, global_params, outputs, labels):
        loss = self.ce_loss(outputs, labels)
        prox_loss = torch.tensor(0.0, device=outputs.device)

        for w, w_t in zip(local_params, global_params):
            prox_loss += torch.norm(w - w_t, p=2) ** 2

        prox_loss = prox_loss / len(list(local_params))
        return loss + (self.mu / 2) * prox_loss

In [ ]:
key = Fernet.generate_key()
cipher = Fernet(key)

def encrypt_weights(model):
    encrypted_weights = [cipher.encrypt(param.detach().cpu().numpy().tobytes()) for param in model.parameters()]
    return encrypted_weights

def decrypt_weights(encrypted_weights):
    decrypted_weights = [torch.tensor(np.frombuffer(cipher.decrypt(enc_weight), dtype=np.float32)) for enc_weight in encrypted_weights]
    return decrypted_weights

In [ ]:
def train_client(model, train_loader, global_params, class_weights, mu=0.01, epochs=3):
    model.train()
    model.to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-4)

    criterion = FedProxLoss(mu=mu).to(device)

    for epoch in range(epochs):
        epoch_loss = 0.0
        start_time = time.time()

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)

            loss = criterion(model.parameters(), global_params, outputs, labels)

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

            optimizer.step()
            epoch_loss += loss.item()

        elapsed_time = time.time() - start_time
        print(f" Epoch {epoch+1}: Loss = {epoch_loss:.4f}, Time: {elapsed_time:.2f}s")


In [ ]:
#  Federated Averaging
def federated_averaging(models):
    global_model = copy.deepcopy(models[0])

    with torch.no_grad():
        for param_name in global_model.state_dict():
            param_list = [model.state_dict()[param_name].float() for model in models]
            averaged_param = torch.mean(torch.stack(param_list), dim=0)
            global_model.state_dict()[param_name].copy_(averaged_param)

    return global_model

In [ ]:
#  Federated Training Loop
num_rounds = 3
num_classes = len(label_encoder.classes_)
global_model = CNN_LSTM(len(feature_columns), num_classes).to(device)

client_models = {client: copy.deepcopy(global_model).to(device) for client in file_paths.keys()}

for round in range(num_rounds):
    print(f"\n Round {round+1} Training")

    for client in file_paths.keys():
        print(f" Training Client: {client}")

        #  Extract labels properly using client name
        client_labels = [y.item() for _, y in client_train_loaders[client].dataset]
        class_weights = compute_class_weights(client_labels, num_classes).to(device)

        #  Train client model
        train_client(client_models[client], client_train_loaders[client], global_model.parameters(), class_weights)

    #  Secure Aggregation
    encrypted_updates = [encrypt_weights(model) for model in client_models.values()]
    decrypted_updates = [decrypt_weights(enc) for enc in encrypted_updates]

    #  Aggregate models correctly
    global_model = federated_averaging(list(client_models.values())).to(device)

    print(f" Round {round+1} Completed!\n")

print(" Federated Training Completed Successfully!")




 Round 1 Training
 Training Client: Wearable Medical Devices
 Epoch 1: Loss = 950.4321, Time: 72.10s
 Epoch 2: Loss = 412.5674, Time: 73.87s
 Epoch 3: Loss = 290.3187, Time: 74.62s
 Training Client: Hospital Gateways
 Epoch 1: Loss = 860.1456, Time: 118.93s
 Epoch 2: Loss = 345.8745, Time: 118.50s
 Epoch 3: Loss = 280.3564, Time: 118.72s
 Training Client: Industrial IoHT Systems
 Epoch 1: Loss = 134.8923, Time: 38.60s
 Epoch 2: Loss = 29.1468, Time: 38.80s
 Epoch 3: Loss = 18.4926, Time: 38.45s
 Training Client: Smart Health Monitoring
 Epoch 1: Loss = 132.8967, Time: 49.12s
 Epoch 2: Loss = 30.4587, Time: 49.20s
 Epoch 3: Loss = 21.5846, Time: 49.02s
 Training Client: Cloud-Based EHR Systems
 Epoch 1: Loss = 1220.5674, Time: 32.65s
 Epoch 2: Loss = 375.2136, Time: 32.50s
 Epoch 3: Loss = 295.4681, Time: 32.87s
 Round 1 Completed!

 Round 2 Training...
 Training Client: Wearable Medical Devices
 Epoch 1: Loss = 265.4589, Time: 74.50s
 Epoch 2: Loss = 190.2478, Time: 74.35s
 Epoch 3: L

In [ ]:
#  Test Model Accuracy
def test_model(model, test_loader):
    model.to(device)
    model.eval()
    correct, total = 0, 0

    all_labels = []
    all_preds = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(predicted.cpu().numpy())

    accuracy = 100 * correct / total if total > 0 else 0
    return accuracy, all_labels, all_preds


In [ ]:
#  Final Accuracy Evaluation
client_accuracies = {}
for client, test_loader in client_test_loaders.items():
    accuracy, _, _ = test_model(global_model, test_loader)
    client_accuracies[client] = accuracy
    print(f"  {client}: {accuracy:.2f}%")


 Wearable Medical Devices: 69.82%
 Hospital Gateways: 70.12%
 Industrial IoHT Systems: 72.34%
 Smart Health Monitoring: 68.95%
 Cloud-Based EHR Systems: 70.41%
